In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
df = pd.read_csv('/home/than/DeepLearning/cxr_concept/CheXzero/checkpoints/dinov2-multi-v1.0_vitb/test_results/padchest_test_results.csv')
# f = df[['Atelectasis_auc', 'Cardiomegaly_auc', 'Consolidation_auc', 'Edema_auc', 'Pleural Effusion_auc']]
df = df.transpose().reset_index().dropna()
df.columns = ['disease', 'value']
df = df.sort_values(by='value', ascending=False).reset_index(drop=True)
df.head(30)
df['value'] = df['value'].astype(float)
df['value'].mean()

,disease,value
0,respiratory distress_auc,0.996474
1,pericardial effusion_auc,0.994920
2,total atelectasis_auc,0.994489
3,pulmonary edema_auc,0.977286
4,pleural effusion_auc,0.974854
5,nsg tube_auc,0.973219
6,pneumomediastinum_auc,0.964400
7,loculated fissural effusion_auc,0.958842
8,hydropneumothorax_auc,0.956932
9,mediastinal shift_auc,0.948022


0.6977557047292215

### Old

In [2]:
files = "/home/than/DeepLearning/CheXzero/checkpoints/pt-imp-v3.0/*.csv"
files = glob(files)
files = sorted(files)
filenames = [os.path.basename(f).split("_")[-1].split('.')[0] for f in files]
cols = ['Atelectasis_auc', 'Cardiomegaly_auc', 'Consolidation_auc', 'Edema_auc', 'Pleural Effusion_auc',
        # 'Pleural Other_auc', 'Support Devices_auc',
        # 'Enlarged Cardiomediastinum_auc', 'Lung Lesion_auc', 'Lung Opacity_auc', 'Pneumonia_auc', 'Pneumothorax_auc'
        ]
average_aucs = [pd.read_csv(f)[cols].mean().values for f in files]
df = pd.DataFrame(average_aucs, columns=cols)
df['filename'] = filenames
df['average_auc'] = df[cols].mean(axis=1)
df.sort_values(by='average_auc', ascending=False, inplace=True)
df.head(10)

,Atelectasis_auc,Cardiomegaly_auc,Consolidation_auc,Edema_auc,Pleural Effusion_auc,filename,average_auc
311,0.769600,0.806762,0.874070,0.911242,0.879136,3800,0.848162
78,0.782293,0.822139,0.862909,0.897981,0.872358,1700,0.847536
200,0.758400,0.836273,0.864025,0.893159,0.883042,2800,0.846980
356,0.807893,0.813998,0.854167,0.886377,0.863856,4200,0.845258
89,0.808213,0.805405,0.842820,0.894665,0.874081,1800,0.845037
334,0.765653,0.825079,0.875186,0.884268,0.872587,4000,0.844555
549,0.823680,0.793080,0.851562,0.877034,0.876379,6400,0.844347
189,0.731520,0.814677,0.886161,0.897830,0.883042,2700,0.842646
367,0.776533,0.812189,0.846726,0.905214,0.869945,4300,0.842121
478,0.758080,0.822252,0.870164,0.891953,0.866958,5300,0.841881


In [3]:
from train import load_clip
import zero_shot
from tqdm import tqdm 
from eval import evaluate 
import clip
import torch
from torchvision.transforms import Compose, Normalize, Resize, InterpolationMode

In [13]:
model = load_clip(model_path="/home/than/DeepLearning/CheXzero/checkpoints/pt-imp-v3.0/checkpoint_3800.pt", 
                  pretrained=False, 
                  context_length=77)

Using cache found in /home/than/.cache/torch/hub/facebookresearch_dinov2_main


Loaded in clip model.


In [5]:
def setup_test(val_cxr_filepath, val_label_path, random_init=True):
    val_labels = ['Atelectasis','Cardiomegaly', 'Consolidation', 'Edema',
                  'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
                  'Lung Opacity', 'No Finding','Pleural Effusion',
                  'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices']

    # Define standard +/- templates for softmax evaluation
    val_templates = [("{}", "no {}")] # Using a tuple pair for softmax eval

    # Load ground truth validation labels
    print(f"Loading validation labels from: {val_label_path}")
    y_true_val = zero_shot.make_true_labels(
        cxr_true_labels_path=val_label_path,
        cxr_labels=val_labels,
        cutlabels=True # bool, if True, will keep columns of ground truth labels that correspond
                       # with the labels inputted through `cxr_labels`. Otherwise, drop the first column and keep remaining.
    )

    # input_resolution = 224 if not random_init else 308 # Adjust based on your model init
    input_resolution = 224  # Standard input resolution for CLIP models
    print(f"Using validation input resolution: {input_resolution}")

    val_transform = Compose([
        Normalize((101.48761, 101.48761, 101.48761), (83.43944, 83.43944, 83.43944)),
        Resize(input_resolution, interpolation=InterpolationMode.BICUBIC),
    ])

    # Create validation dataset
    print(f"Loading validation CXR data from: {val_cxr_filepath}")
    val_dataset = zero_shot.CXRTestDataset(
        img_path=val_cxr_filepath,
        transform=val_transform,
    )

    # Create validation dataloader
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=64,
        shuffle=False, # No need to shuffle for validation
        num_workers=2, # Adjust based on your system
        pin_memory=True
    )

    print("Validation setup complete.")
    return val_loader, y_true_val, val_labels, val_templates, input_resolution

In [6]:
val_loader, y_true_val, val_labels, val_templates, _ = setup_test(
    val_cxr_filepath="data/chexpert_test.h5",
    val_label_path="data/chexpert_test.csv",
)

Loading validation labels from: data/chexpert_test.csv
Using validation input resolution: 224
Loading validation CXR data from: data/chexpert_test.h5
Validation setup complete.


In [8]:
model = model.to('cuda')
_ = model.eval()
context_length = 77
device = 'cuda'
pos_template, neg_template = val_templates[0]

with torch.no_grad():
    pos_texts = [pos_template.format(c) for c in val_labels]
    neg_texts = [neg_template.format(c) for c in val_labels]
    pos_tokens = clip.tokenize(pos_texts, context_length).to(device)
    neg_tokens = clip.tokenize(neg_texts, context_length).to(device)
    pos_features = model.encode_text(pos_tokens)
    neg_features = model.encode_text(neg_tokens)
    pos_features /= pos_features.norm(dim=-1, keepdim=True)
    neg_features /= neg_features.norm(dim=-1, keepdim=True)

all_img_feats = []
with torch.no_grad():
    for data in tqdm(val_loader, desc="Validation Inference"):
        imgs = data['img'].to(device)
        feats = model.encode_image(imgs)
        feats /= feats.norm(dim=-1, keepdim=True)
        all_img_feats.append(feats.cpu())

img_feats_cat = torch.cat(all_img_feats).to(device)
logits_pos = img_feats_cat @ pos_features.T
logits_neg = img_feats_cat @ neg_features.T
probs = torch.exp(logits_pos) / (torch.exp(logits_pos) + torch.exp(logits_neg))
y_pred_val = probs.cpu().numpy()
val_results_df = evaluate(y_pred_val, y_true_val, val_labels)

Validation Inference:   0%|          | 0/8 [00:00<?, ?it/s]

Validation Inference: 100%|██████████| 8/8 [00:00<00:00,  9.52it/s]


In [9]:
val_results_df
cols = ['Atelectasis_auc', 'Cardiomegaly_auc', 'Consolidation_auc', 'Edema_auc', 'Pleural Effusion_auc']
val_results_df = val_results_df[cols]
val_results_df
val_results_df.mean().mean()

,Atelectasis_auc,Cardiomegaly_auc,Consolidation_auc,Edema_auc,Enlarged Cardiomediastinum_auc,Fracture_auc,Lung Lesion_auc,Lung Opacity_auc,No Finding_auc,Pleural Effusion_auc,Pleural Other_auc,Pneumonia_auc,Pneumothorax_auc,Support Devices_auc
0,0.787403,0.893812,0.893038,0.906915,0.894145,0.712929,0.680767,0.916362,0.07862,0.930908,0.620968,0.801729,0.629667,0.671989


,Atelectasis_auc,Cardiomegaly_auc,Consolidation_auc,Edema_auc,Pleural Effusion_auc
0,0.787403,0.893812,0.893038,0.906915,0.930908


0.8824149100397852

### PadChest evaluation

In [5]:
df_padchest = pd.read_csv("csvs/padchest_test.csv")
df_padchest = df_padchest[df_padchest['is_test'] == True]
df_padchest.head()
len(df_padchest)
# df_padchest.to_csv("data/padchest_test.csv", index=False)

,ImageID,total atelectasis,reservoir central venous catheter,lipomatosis,subacromial space narrowing,surgery neck,hilar congestion,suboptimal study,miliary opacities,mediastinal shift,...,pneumothorax,hilar enlargement,cardiomegaly,hiatal hernia,dextrocardia,hydropneumothorax,aortic endoprosthesis,name,Path,is_test
0,113855343774216031107737439268243531979_3k951l...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,113855343774216031107737439268243531979_3k951l...,/home/than/padchest/test/113855343774216031107...,True
1,258055173326723517262722952856353517225_gxjwwm...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,258055173326723517262722952856353517225_gxjwwm...,/home/than/padchest/test/258055173326723517262...,True
2,164248849740406851039799839958404888661_etstxt...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,164248849740406851039799839958404888661_etstxt...,/home/than/padchest/test/164248849740406851039...,True
3,200091420165336548849526592472869689994_n0exds...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,200091420165336548849526592472869689994_n0exds...,/home/than/padchest/test/200091420165336548849...,True
4,14968019924555248865694512726537711769_3r162a.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,14968019924555248865694512726537711769_3r162a.png,/home/than/padchest/test/149680199245552488656...,True


7943

In [6]:
disease_labels = df_padchest.columns[1:-3].tolist()

In [7]:
def setup_test(val_cxr_filepath, val_label_path, random_init=True):
    val_labels = [d.lower() for d in disease_labels]

    # Define standard +/- templates for softmax evaluation
    val_templates = [("{}", "no {}")] # Using a tuple pair for softmax eval

    # Load ground truth validation labels
    print(f"Loading validation labels from: {val_label_path}")
    y_true_val = zero_shot.make_true_labels(
        cxr_true_labels_path=val_label_path,
        cxr_labels=val_labels,
        cutlabels=True # bool, if True, will keep columns of ground truth labels that correspond
                       # with the labels inputted through `cxr_labels`. Otherwise, drop the first column and keep remaining.
    )

    input_resolution = 224 # Adjust based on your model init
    print(f"Using validation input resolution: {input_resolution}")

    val_transform = Compose([
        Normalize((101.48761, 101.48761, 101.48761), (83.43944, 83.43944, 83.43944)),
        Resize(input_resolution, interpolation=InterpolationMode.BICUBIC),
    ])

    # Create validation dataset
    print(f"Loading validation CXR data from: {val_cxr_filepath}")
    val_dataset = zero_shot.CXRTestDataset(
        img_path=val_cxr_filepath,
        transform=val_transform,
    )

    # Create validation dataloader
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=64,
        shuffle=False, # No need to shuffle for validation
        num_workers=2, # Adjust based on your system
        pin_memory=True
    )

    print("Validation setup complete.")
    return val_loader, y_true_val, val_labels, val_templates, input_resolution

In [8]:
val_loader, y_true_val, val_labels, val_templates, _ = setup_test(
    val_cxr_filepath="data/padchest_test.h5",
    val_label_path="data/padchest_test.csv",
)

Loading validation labels from: data/padchest_test.csv
Using validation input resolution: 224
Loading validation CXR data from: data/padchest_test.h5
Validation setup complete.


In [14]:
model = model.to('cuda')
_ = model.eval()
context_length = 77
device = 'cuda'
pos_template, neg_template = val_templates[0]

with torch.no_grad():
    pos_texts = [pos_template.format(c) for c in val_labels]
    neg_texts = [neg_template.format(c) for c in val_labels]
    pos_tokens = clip.tokenize(pos_texts, context_length).to(device)
    neg_tokens = clip.tokenize(neg_texts, context_length).to(device)
    pos_features = model.encode_text(pos_tokens)
    neg_features = model.encode_text(neg_tokens)
    pos_features /= pos_features.norm(dim=-1, keepdim=True)
    neg_features /= neg_features.norm(dim=-1, keepdim=True)

all_img_feats = []
with torch.no_grad():
    for data in tqdm(val_loader, desc="Validation Inference"):
        imgs = data['img'].to(device)
        feats = model.encode_image(imgs)
        feats /= feats.norm(dim=-1, keepdim=True)
        all_img_feats.append(feats.cpu())

img_feats_cat = torch.cat(all_img_feats).to(device)
logits_pos = img_feats_cat @ pos_features.T
logits_neg = img_feats_cat @ neg_features.T
probs = torch.exp(logits_pos) / (torch.exp(logits_pos) + torch.exp(logits_neg))
y_pred_val = probs.cpu().numpy()
val_results_df = evaluate(y_pred_val, y_true_val, val_labels)

Validation Inference: 100%|██████████| 125/125 [00:16<00:00,  7.43it/s]


In [16]:
print(val_results_df.mean().sort_values(ascending=False).dropna().mean())
print(val_results_df.mean().sort_values(ascending=False).dropna().head(50))
val_results_df['pleural effusion_auc']

0.6923363595788727
lepidic adenocarcinoma_auc                         0.985936
respiratory distress_auc                           0.983631
pulmonary edema_auc                                0.974421
pericardial effusion_auc                           0.970487
pleural effusion_auc                               0.969658
hydropneumothorax_auc                              0.967573
cyst_auc                                           0.959325
central venous catheter via jugular vein_auc       0.952195
heart insufficiency_auc                            0.937836
mediastinal shift_auc                              0.937264
central venous catheter via subclavian vein_auc    0.936663
pulmonary fibrosis_auc                             0.934736
atypical pneumonia_auc                             0.933756
tracheostomy tube_auc                              0.916529
pulmonary mass_auc                                 0.916108
abscess_auc                                        0.914757
loculated pleural eff

0    0.969658
Name: pleural effusion_auc, dtype: float64